Rough sketch of a proposal

- probably wrong for conjugate
- only optimized allocations and contract! (tensoradd! and tensortrace! were not modified)
- probably fails for non-symmetric tensormaps

In [20]:
using Revise, TensorKit, TensorOperations, BenchmarkTools, Strided, TupleTools, AllocationKit, MPSKitExperimental

In [21]:
import LinearAlgebra
LinearAlgebra.BLAS.set_num_threads(1)
Strided.set_num_threads(1)

1

In [22]:
using Base.Threads
nthreads()

8

# practical benchmarks

In [23]:
@tightloop_tensor ac_eff y[-1 -2;-3] := le[-1 2;1]*O[2 -2;3 4]*x[1 3;5]*re[5 4;-3]
@tightloop_tensor ac_eff_malloc allocator=malloc y[-1 -2;-3] := le[-1 2;1]*O[2 -2;3 4]*x[1 3;5]*re[5 4;-3]

In [24]:
virtspace = Rep[SU₂](i => 20 for i in 0:10);
ospace = Rep[SU₂](0 => 5,1 => 2);
pspace = Rep[SU₂](1 => 1);

t_le = TensorMap(rand,ComplexF64,virtspace*ospace',virtspace);
t_re = TensorMap(rand,ComplexF64,virtspace*ospace,virtspace);
t_ac = TensorMap(rand,ComplexF64,virtspace*pspace,virtspace);
t_o = TensorMap(rand,ComplexF64,ospace*pspace,pspace*ospace);

factory = ac_eff(le = (typeof(t_le),space(t_le)),re = (typeof(t_re),space(t_re)),O = (typeof(t_o),space(t_o)),x = (typeof(t_ac),space(t_ac)));
factory_malloc = ac_eff_malloc(le = (typeof(t_le),space(t_le)),re = (typeof(t_re),space(t_re)),O = (typeof(t_o),space(t_o)),x = (typeof(t_ac),space(t_ac)));

In [25]:
@benchmark $factory_malloc(le = $t_le, re=$t_re, x = $t_ac, O = $t_o)

BenchmarkTools.Trial: 1331 samples with 1 evaluation.
 Range (min … max):  1.957 ms …  11.105 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     3.761 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   3.724 ms ± 925.724 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

   ▂                 ▁▆██▅▇▆▅▅▆▄▄▂▂▄▂                          
  ▇██▇▇█▇▇▆▄▃▃▂▂▂▂▁▂▄█████████████████▆▇▇▆▆▅▆▅▄▃▃▃▃▃▃▃▃▂▃▂▂▁▃ ▅
  1.96 ms         Histogram: frequency by time        6.04 ms <

 Memory estimate: 195.11 KiB, allocs estimate: 155.

In [26]:
@benchmark $factory(le = $t_le, re=$t_re, x = $t_ac, O = $t_o)

BenchmarkTools.Trial: 1472 samples with 1 evaluation.
 Range (min … max):  2.563 ms …  10.595 ms  ┊ GC (min … max): 0.00% … 64.22%
 Time  (median):     3.019 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   3.371 ms ± 908.011 μs  ┊ GC (mean ± σ):  7.25% ± 12.39%

    ▆█▇█▅▄▁                                                    
  ▂▅███████▇█▆▄▃▃▃▃▃▂▂▃▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▂▁▂▂▂▂▂▂▃▁▁▂▂▁▂▂▁▁▁▁▁ ▂
  2.56 ms         Histogram: frequency by time         6.3 ms <

 Memory estimate: 8.28 MiB, allocs estimate: 148.

In [27]:
function slowcontract(;le=nothing,re=nothing,x=nothing,O=nothing)
    @tensor y[-1 -2;-3] := le[-1 2;1]*O[2 -2;3 4]*x[1 3;5]*re[5 4;-3]
end
@benchmark slowcontract(le = $t_le, re=$t_re, x = $t_ac, O = $t_o)

BenchmarkTools.Trial: 1012 samples with 1 evaluation.
 Range (min … max):  3.786 ms … 13.277 ms  ┊ GC (min … max): 0.00% … 62.65%
 Time  (median):     4.432 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   4.920 ms ±  1.398 ms  ┊ GC (mean ± σ):  7.71% ± 12.51%

    ▅▆██▆▄▂▁                                                  
  ▃█████████▆▇▆▄▄▃▄▃▂▃▂▂▂▂▂▂▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▂▂▂▃▃▃▃▄▃▃▃▄▂▂▂▂ ▃
  3.79 ms        Histogram: frequency by time        9.31 ms <

 Memory estimate: 9.62 MiB, allocs estimate: 16707.

In [28]:
norm(slowcontract(le = t_le, re=t_re, x = t_ac, O = t_o)- factory(le = t_le, re=t_re, x = t_ac, O = t_o))

0.0

In [29]:
virtspace = Rep[SU₂×U₁×ℤ₂]((i,j,b) => 10 for i in 0:5,j in -3:3,b in 0:1);
ospace = Rep[SU₂×U₁×ℤ₂]((i,j,b) => 2 for i in 0:1,j in -1:1,b in 0:1);
pspace = Rep[SU₂×U₁×ℤ₂]((i,j,b) => 1 for i in 1:1,j in 0:0,b in 0:0);

t_le = TensorMap(rand,ComplexF64,virtspace*ospace',virtspace);
t_re = TensorMap(rand,ComplexF64,virtspace*ospace,virtspace);
t_ac = TensorMap(rand,ComplexF64,virtspace*pspace,virtspace);
t_o = TensorMap(rand,ComplexF64,ospace*pspace,pspace*ospace);

factory = ac_eff(le = (typeof(t_le),space(t_le)),re = (typeof(t_re),space(t_re)),O = (typeof(t_o),space(t_o)),x = (typeof(t_ac),space(t_ac)));

In [30]:
@benchmark $factory(le = $t_le, re=$t_re, x = $t_ac, O = $t_o)

BenchmarkTools.Trial: 165 samples with 1 evaluation.
 Range (min … max):  26.480 ms … 40.873 ms  ┊ GC (min … max): 0.00% … 31.47%
 Time  (median):     30.252 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   30.396 ms ±  2.521 ms  ┊ GC (mean ± σ):  4.15% ±  5.08%

      ▄▆█▃▃▄▄▃▄     ▆ ▄▁▃  ▆ ▆▃▁   ▁▄▆                         
  ▄▄▁▇█████████▇▇▄▆▁█▄███▆▇█▇███▁▆▆███▇▆▄▆▁▄▄▇▁▄▁▁▁▄▄▄▁▄▁▄▁▁▄ ▄
  26.5 ms         Histogram: frequency by time        36.9 ms <

 Memory estimate: 53.59 MiB, allocs estimate: 691.

In [31]:
@benchmark slowcontract(le = $t_le, re=$t_re, x = $t_ac, O = $t_o)

BenchmarkTools.Trial: 2 samples with 1 evaluation.
 Range (min … max):  4.126 s …   4.229 s  ┊ GC (min … max): 0.00% … 1.33%
 Time  (median):     4.178 s              ┊ GC (median):    0.67%
 Time  (mean ± σ):   4.178 s ± 73.138 ms  ┊ GC (mean ± σ):  0.67% ± 0.94%

  █                                                       █  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  4.13 s         Histogram: frequency by time        4.23 s <

 Memory estimate: 237.95 MiB, allocs estimate: 1928698.

In [32]:
norm(slowcontract(le = t_le, re=t_re, x = t_ac, O = t_o)- factory(le = t_le, re=t_re, x = t_ac, O = t_o))

0.0

# multithreaded benchmark

gc becomes a severe bottleneck with many julia threads

In [33]:
virtspace = Rep[SU₂](0=>50, 1=> 30, 2=> 20, 3=>10, 4=> 5);
ospace = Rep[SU₂](0=>2, 1 => 2);
pspace = Rep[SU₂](1 => 1);


In [34]:
numblocks = 100;
const leftenvs = [TensorMap(rand,ComplexF64,virtspace*ospace',virtspace) for i in 1:numblocks];
const rightenvs = [TensorMap(rand,ComplexF64,virtspace*ospace,virtspace) for i in 1:numblocks];
const mpos = [TensorMap(rand,ComplexF64,ospace*pspace,pspace*ospace) for i in 1:numblocks];

ac = TensorMap(rand,ComplexF64,virtspace*pspace,virtspace);

In [38]:
using FLoops

t_le = leftenvs[1]
t_re = rightenvs[1]
t_o = mpos[1]

const fast_factory = ac_eff_malloc(le = (typeof(t_le),space(t_le)),re = (typeof(t_re),space(t_re)),O = (typeof(t_o),space(t_o)),x = (typeof(ac),space(ac)))

function fastfun(x)
    @floop for (l,m,r) in zip(leftenvs,mpos,rightenvs)
        y = fast_factory(le = l,re = r, O = m, x = x)
        @reduce s += y
    end
    s
end

function slowfun(x)
    @floop for (le,O,re) in zip(leftenvs,mpos,rightenvs)
        @tensor y[-1 -2;-3] := le[-1 2;1]*O[2 -2;3 4]*x[1 3;5]*re[5 4;-3]
        @reduce s += y
    end
    s
end

slowfun (generic function with 1 method)

In [39]:
@benchmark fastfun(ac)

BenchmarkTools.Trial: 188 samples with 1 evaluation.
 Range (min … max):  16.725 ms … 64.831 ms  ┊ GC (min … max): 0.00% … 60.21%
 Time  (median):     24.791 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   26.597 ms ±  7.406 ms  ┊ GC (mean ± σ):  2.26% ±  7.46%

      ▁▆█▅▃▆▆▃                                                 
  ▃▄▅▇████████▆█▆▇▅▆▅▄▆▅▄▄▃▆▄▁▄▁▁▃▁▃▁▁▁▁▁▁▁▁▁▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃ ▃
  16.7 ms         Histogram: frequency by time        63.1 ms <

 Memory estimate: 19.26 MiB, allocs estimate: 12458.

In [40]:
@benchmark slowfun(ac)

BenchmarkTools.Trial: 72 samples with 1 evaluation.
 Range (min … max):  56.502 ms … 94.379 ms  ┊ GC (min … max):  0.00% … 17.99%
 Time  (median):     72.025 ms              ┊ GC (median):    17.83%
 Time  (mean ± σ):   70.022 ms ±  9.628 ms  ┊ GC (mean ± σ):  10.78% ±  9.58%

  ▅ █▅   ▅  ▂           ▂  ▂ ▅▅▅ ▂ ▅                           
  ████▅▁▁█▁▅█▁█▅▁▅▅▅▁▅▁▅█▁▁█████▁█▁█▁▁▁▅▅█▅▅▁▅▁▁▅▅▁▁▅▁▁▁▁▅▁▁▅ ▁
  56.5 ms         Histogram: frequency by time        92.3 ms <

 Memory estimate: 345.16 MiB, allocs estimate: 598358.